In [1]:
import transformers

print(transformers.__version__)


4.35.2


In [3]:
task = "sst2"
'''
The pretrained model is being pulled
'''
model_checkpoint = "ayushmodi12/pretrained_bert_scratch"
batch_size = 16

In [4]:
!pip install datasets transformers

In [5]:
from datasets import load_dataset, load_metric
'''
Using the dataset of sst2 loaded in transformers dataset
'''
dataset=load_dataset("glue","sst2")
metric=load_metric('glue','sst2')


<ipython-input-5-0b85de7ba4d7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric('glue','sst2')


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [6]:
from transformers import AutoTokenizer
'''
Defining the tokenizer
'''
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}


In [8]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: hide new secretions from the parental units 


In [9]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], [101, 3397, 2053, 15966, 1010, 2069, 4450, 2098, 18201, 2015, 102], [101, 2008, 7459, 2049, 3494, 1998, 10639, 2015, 2242, 2738, 3376, 2055, 2529, 3267, 102], [101, 3464, 12580, 8510, 2000, 3961, 1996, 2168, 2802, 102], [101, 2006, 1996, 5409, 7195, 1011, 1997, 1011, 1996, 1011, 11265, 17811, 18856, 17322, 2015, 1996, 16587, 2071, 2852, 24225, 2039, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [10]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ayushmodi12/pretrained_bert_scratch and are newly initialized: ['bert.pooler.dense.bias', 'classifier.weight', 'bert.pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
! pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.0 MB/s eta 0:00:00


In [12]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [14]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [15]:
import numpy as np

trainer.train()



You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.695900,0.694548,0.490826
2,0.694500,0.721894,0.509174
3,0.696300,0.702839,0.509174
4,0.699000,0.693834,0.509174
5,0.693800,0.704753,0.509174


TrainOutput(global_step=21050, training_loss=0.7025486976460437, metrics={'train_runtime': 2850.9766, 'train_samples_per_second': 118.116, 'train_steps_per_second': 7.383, 'total_flos': 6089631376322880.0, 'train_loss': 0.7025486976460437, 'epoch': 5.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.7218940258026123,
 'eval_accuracy': 0.5091743119266054,
 'eval_runtime': 2.5368,
 'eval_samples_per_second': 343.74,
 'eval_steps_per_second': 21.681,
 'epoch': 5.0}

In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [19]:

'''
Pushing the model to hugging face as mentioned in the question
'''
trainer.push_to_hub()

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1700393387.1b1cabfc6ce8.1453.0:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

events.out.tfevents.1700396688.1b1cabfc6ce8.1453.1:   0%|          | 0.00/417 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

'https://huggingface.co/CodeinJax/pretrained_bert_scratch-finetuned-sst2/tree/main/'

In [20]:
# Use a pipeline as a high-level helper
'''
Pulling the pushed model from hugging face and then evaluating the metrics
'''
from transformers import pipeline

pipe = pipeline("text-classification", model="CodeinJax/pretrained_bert_scratch-finetuned-sst2")

config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [21]:
ds_eval=encoded_dataset[validation_key]

In [22]:
from tqdm import tqdm
inputs=ds_eval['sentence']
outp=ds_eval['label']
predicted=[]
for i in tqdm(inputs):
  predicted.append(pipe(i))

100%|██████████| 872/872 [01:56<00:00,  7.51it/s]


In [23]:
def funn(xx):
  if(xx[0]['label']=='LABEL_0'):
    return 0
  else:
    return 1


In [24]:
predicted2=[funn(lbl) for lbl in predicted]

In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
'''
Evaluating the model on the data and evaluating the metrics
'''
precision=precision_score(outp,predicted2)
recall=recall_score(outp,predicted2)
f1=f1_score(outp,predicted2)
accuracy=accuracy_score(outp,predicted2)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 0.5091743119266054
Recall: 1.0
F1 Score: 0.6747720364741641
Accuracy: 0.5091743119266054


In [26]:
! pip install torchinfo

In [28]:
# Load model directly
'''
Printing the model parameters, the answers to the questions can be found in the report
'''
from torchinfo import summary
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("CodeinJax/pretrained_bert_scratch-finetuned-sst2")
print("<<<<<<<<<<<<<<<<<<<<Printing Layer Shapes>>>>>>>>>>>>>>>>>>>>")
print("<<<<<<<<<<<<<<<<<<<<Bert-base Shapes>>>>>>>>>>>>>>>>>>>>")
print(summary(model))
print("<<<<<<<<<<<<<<<<<<<<Classifier (Linear) Layer>>>>>>>>>>>>>>>>>>>>")
print(summary(model.classifier))

<<<<<<<<<<<<<<<<<<<<Printing Layer Shapes>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<Bert-base Shapes>>>>>>>>>>>>>>>>>>>>
Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2    

In [31]:
predicted

[[{'label': 'LABEL_1', 'score': 0.6273288726806641}],
 [{'label': 'LABEL_1', 'score': 0.6273292899131775}],
 [{'label': 'LABEL_1', 'score': 0.6273230910301208}],
 [{'label': 'LABEL_1', 'score': 0.6273302435874939}],
 [{'label': 'LABEL_1', 'score': 0.6273238062858582}],
 [{'label': 'LABEL_1', 'score': 0.627318799495697}],
 [{'label': 'LABEL_1', 'score': 0.6273314952850342}],
 [{'label': 'LABEL_1', 'score': 0.6273194551467896}],
 [{'label': 'LABEL_1', 'score': 0.6273292899131775}],
 [{'label': 'LABEL_1', 'score': 0.6273345947265625}],
 [{'label': 'LABEL_1', 'score': 0.6273308396339417}],
 [{'label': 'LABEL_1', 'score': 0.6273331642150879}],
 [{'label': 'LABEL_1', 'score': 0.6273215413093567}],
 [{'label': 'LABEL_1', 'score': 0.6273278594017029}],
 [{'label': 'LABEL_1', 'score': 0.6273221969604492}],
 [{'label': 'LABEL_1', 'score': 0.6273324489593506}],
 [{'label': 'LABEL_1', 'score': 0.6273331642150879}],
 [{'label': 'LABEL_1', 'score': 0.6273230910301208}],
 [{'label': 'LABEL_1', 'score